# Volatility Model Calibration

This notebook demonstrates **Time-Adjusted Wing Model calibration** with a streamlined approach.

## Features
- **Single Model Focus**: Time-Adjusted Wing Model only
- **Configuration-Driven**: Respects config settings
- **Simplified Flow**: Reduced complexity and cleaner cells
- **Visual Results**: Clear performance visualization

## 1. Setup

In [1]:
# Essential imports
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project setup
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)
# Configuration
config = {
    'date': '20240229',
    'option_expiry': '8MAR24',
    'snapshot_time': '2024-02-29T20:22:00'
}

print(f"📋 Using expiry: {config['option_expiry']}, date: {config['date']}")

📋 Using expiry: 8MAR24, date: 20240229


## 2. Data Loading

In [2]:
# Load market data
from utils.volatility_fitter.processed_data_loader import create_snapshot_option_chain, load_baseoffset_results, load_option_market_data
from utils.volatility_fitter.volatility_calculator import get_option_chains
import polars as pl

date_str = config['date']
snapshot_time = datetime.strptime(config['snapshot_time'], "%Y-%m-%dT%H:%M:%S")
my_expiry = config['option_expiry']

# Load and process data
df_baseoffset = load_baseoffset_results(date_str)
df_option_md = load_option_market_data(date_str)
df_snapshot_md = create_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)
df_option_chain = get_option_chains(df_snapshot_md, my_expiry, snapshot_time)

print(f"📊 Loaded {len(df_option_chain)} option contracts for {my_expiry}")

Available expiries: ['2MAR24', '27SEP24', '8MAR24', '15MAR24', '26APR24', '28JUN24', '29MAR24', '3MAR24', '29FEB24', '31MAY24', '22MAR24', '27DEC24', '1MAR24']
📊 Loaded 41 option contracts for 8MAR24


In [3]:
df_option_md.filter(pl.col('symbol')=='BTC-29FEB24-64500-C').head()

symbol,timestamp,expiry,strike,bid_size,bid_price,ask_price,ask_size,S,expiry_ts,is_option,is_call,tau
str,datetime[ns],str,i64,f64,f64,f64,f64,f64,datetime[ns],bool,bool,f64
"""BTC-29FEB24-64500-C""",2024-02-29 00:11:30,"""29FEB24""",64500,8.8,0.001,0.0016,24.5,62233.56,2024-02-29 08:00:00,true,true,0.000891
"""BTC-29FEB24-64500-C""",2024-02-29 00:12:00,"""29FEB24""",64500,6.0,0.0009,0.0015,26.8,62166.15,2024-02-29 08:00:00,true,true,0.00089
"""BTC-29FEB24-64500-C""",2024-02-29 00:12:30,"""29FEB24""",64500,6.0,0.0009,0.0015,26.8,62128.42,2024-02-29 08:00:00,true,true,0.000889
"""BTC-29FEB24-64500-C""",2024-02-29 00:13:00,"""29FEB24""",64500,7.4,0.0008,0.0014,36.9,62054.47,2024-02-29 08:00:00,true,true,0.000889
"""BTC-29FEB24-64500-C""",2024-02-29 00:13:30,"""29FEB24""",64500,2.8,0.001,0.0014,36.9,62079.06,2024-02-29 08:00:00,true,true,0.000888


In [4]:
# Retrieve interest rate``
rate = df_baseoffset.filter(pl.col('expiry')==my_expiry, pl.col('timestamp')==snapshot_time)['r'][0]
print(f"interest rate for expiry {my_expiry} @ {snapshot_time} = {rate:.3f}")

interest rate for expiry 8MAR24 @ 2024-02-29 20:22:00 = 0.314


In [5]:
# Process option data
import numpy as np
from utils.pricer.option_constraints import tighten_option_spreads_separate_columns
from utils.reporting.html_table_generator import generate_price_comparison_table
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks
from IPython.display import HTML, display

# Tighten spreads and calculate volatilities
volume_threshold = 1.1  # Minimum volume to consider for spread tightening
tightened_option_chain = tighten_option_spreads_separate_columns(df_option_chain, volume_threshold=volume_threshold, interest_rate=rate)
df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=rate)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)

display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px", volume_threshold=volume_threshold)))

# Extract key parameters
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]
strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
market_vols = (df_option_with_vola_and_greeks['midVola']/100).to_list()
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()
weight_arr = np.array(market_vegas)/max(market_vegas)

print(f"💰 Forward: ${forward_price:,.2f}, ⏰ Time to expiry: {time_to_expiry:.4f} years")
print(f"📈 {len(strikes_list)} strikes from ${min(strikes_list):,.0f} to ${max(strikes_list):,.0f}")

📝 Generating HTML price comparison table for the tightened bid/ask spread on call and put options... (Volume Threshold: 1.1)


💰 Forward: $62,343.55, ⏰ Time to expiry: 0.0205 years
📈 41 strikes from $42,000 to $76,000


In [6]:
df_option_with_vola_and_greeks.head()

timestamp,bq0_C,bp0_C,bp0_C_usd,ap0_C_usd,ap0_C,aq0_C,strike,bq0_P,bp0_P,bp0_P_usd,ap0_P_usd,ap0_P,aq0_P,S,F,expiry,tau,r,bidVola_C,askVola_C,bidVola_P,askVola_P,bidVola,askVola,midVola,volSpread,vega,call_delta
datetime[ns],f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-02-29 20:22:00,6.4,0.316,19604.26,20416.51,0.329093,6.4,42000,7.5,0.0003,18.61,31.02,0.0005,7.9,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,160.563226,112.235346,119.507368,112.24,119.51,115.88,7.27,1.7,0.99
2024-02-29 20:22:00,6.4,0.284,17619.02,18429.35,0.297062,6.4,44000,0.2,0.0005,31.02,43.43,0.0007,10.2,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,144.461441,106.830169,111.8134,106.83,111.81,109.32,4.98,2.49,0.98
2024-02-29 20:22:00,6.3,0.268,16626.4,17435.78,0.281046,6.3,45000,3.4,0.0005,31.02,49.63,0.0008,17.4,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,136.625904,100.669673,107.453753,100.67,107.45,104.06,6.78,2.74,0.98
2024-02-29 20:22:00,6.3,0.252,15633.78,16442.2,0.265031,6.3,46000,3.1,0.0006,37.22,55.83,0.0009,16.3,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,128.917551,96.998608,102.856956,97.0,102.86,99.93,5.86,3.17,0.98
2024-02-29 20:22:00,6.3,0.2365,14672.18,15448.62,0.249015,6.3,47000,4.0,0.0008,49.63,68.24,0.0011,15.3,62038.81,62343.55,"""8MAR24""",0.020506,0.3142,2.36,121.327405,94.790857,99.556841,94.79,99.56,97.18,4.77,3.9,0.98


In [7]:
import plotly.graph_objects as go

fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=strikes_list,
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=strikes_list,
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Model Calibration

In [8]:
# Updated imports - using specific calibrators directly
from utils.volatility_fitter.calibrators import GlobalVolatilityCalibrator
from utils.volatility_fitter.calibrators.local_calibrator import LocalVolatilityCalibrator  # Time-Adjusted Wing Model Calibration
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel
from utils.volatility_fitter.time_adjusted_wing_model import create_time_adjusted_wing_model_from_result

# Load configuration
from config.config_loader import load_config
vol_config = load_config()

print("🔧 Configuration loaded")
print("✅ Time-Adjusted Wing Model: ENABLED")

🔧 Configuration loaded
✅ Time-Adjusted Wing Model: ENABLED


### 3.1 Initial Guess

In [9]:
# Define loss value externally and initial parameters
import numpy as np
from utils.volatility_fitter.calibration_result import CalibrationResult
from utils.volatility_fitter.wing_model.wing_model_parameters import WingModelParameters


def find_objective_loss_value(my_calibrator: GlobalVolatilityCalibrator|LocalVolatilityCalibrator, 
                              model_params: TimeAdjustedWingModel, strikes_list: list[float], 
                              market_vols: list[float], market_vegas: list[float], weights: list[float]) -> float:
    return float(my_calibrator._objective_function(
        x=np.array(model_params.get_fitted_vol_parameter()), initial_params=model_params, 
        param_names=model_params.get_parameter_names(), strikes=strikes_list, market_volatilities=market_vols, 
        market_vegas=market_vegas, enforce_arbitrage_free=True, weights=weights))
    
# Initial parameters
initial_guess = [0.4991, -1.7540, 1.29318, 1.86, -0.093005, 1.4, 3.0, 4.87]
calibration_results: list[str, WingModelParameters, float, str] = []
my_local_calibrator = LocalVolatilityCalibrator(model_class=TimeAdjustedWingModel, enable_bounds=True)

my_initial_wing_model = create_time_adjusted_wing_model_from_result(result=initial_guess,
    forward_price=forward_price, ref_price=forward_price, time_to_expiry=time_to_expiry)

my_initial_result = CalibrationResult(
    success=False, optimization_method="Initial Setup", parameters=my_initial_wing_model, 
    error=find_objective_loss_value(my_local_calibrator, my_initial_wing_model, strikes_list, market_vols, market_vegas, weight_arr), 
    message="Initial parameters before calibration")

calibration_results.append(my_initial_result)
print(f"🎯 Initial result: {my_initial_result}")

🎯 Initial result: CalibrationResult(
  success=False,
  method='Initial Setup',
  parameters=WingModelParameters(vr=0.4991, sr=-1.7540, pc=1.2932, cc=1.8600, dc=-0.0930, uc=1.4000, dsm=3.0000, usm=4.8700),
  error=7.279213,
  message='Initial parameters before calibration'
  optimisation_result=None
  time_elapsed=0.000000 seconds
)


### 3.2 Local Optimizer (SLSQP and L-BFGS-B) - both need to provide an initial guess of the parameter

In [10]:
#### CALIBRATION OF TIME-ADJUSTED WING MODEL ####
methods = ["SLSQP", "L-BFGS-B"]
if vol_config.time_adjusted_wing_model_enabled:
    print(f"🚀 Calibrating Time-Adjusted Wing Model with optimisation methods {methods}")

    for method_name in methods:
        
        # Create appropriate calibrator based on method
        my_local_calibrator = LocalVolatilityCalibrator(model_class=TimeAdjustedWingModel, method=method_name, enable_bounds=True)
        my_result = my_local_calibrator.calibrate(my_initial_wing_model, strikes_list, market_vols, market_vegas, my_initial_wing_model.get_parameter_bounds(),  # Use the bounds from parameter object
            True, None, weight_arr)
        print(f"📍 Using LocalVolatilityCalibrator for {method_name}")
                    
        calibration_results.append(my_result)
        print(f"Loss: {my_result.error:.8f}, Success: {my_result.success}, Message: {my_result.message}, {my_result.parameters}")
    print("✅ Calibration complete")

🚀 Calibrating Time-Adjusted Wing Model with optimisation methods ['SLSQP', 'L-BFGS-B']
📍 Using LocalVolatilityCalibrator for SLSQP
Loss: 0.13390527, Success: True, Message: Optimization successful, WingModelParameters(vr=0.7004, sr=0.0404, pc=1.1024, cc=0.1649, dc=-0.0102, uc=1.4000, dsm=2.7866, usm=4.8700)
📍 Using LocalVolatilityCalibrator for L-BFGS-B
Loss: 0.13388987, Success: True, Message: Optimization successful, WingModelParameters(vr=0.7004, sr=0.0409, pc=1.2558, cc=0.1638, dc=-0.0100, uc=1.4000, dsm=2.8303, usm=4.8700)
✅ Calibration complete


### 3.3 Glocal Optimizer (Differential Evolution) - dun require the initial guess as it will find the global minimum

In [11]:
# Differential Evolution - Global Optimization
if vol_config.time_adjusted_wing_model_enabled :
    print("🧬 Running Differential Evolution (Global Optimization)...")
    
    try:
        # Create DE calibrator with specific settings for performance
        my_global_calibrator = GlobalVolatilityCalibrator(model_class=TimeAdjustedWingModel, enable_bounds=True, workers=5)
        
        param_bound =\
        [(0.69, 0.72),
        (-1.0, 1.0),
        (0.01, 1.0),
        (0.01, 1.0),
        (-5.0, -0.1),
        (0.1, 5.0),
        (1.0, 5.0),
        (1.0, 5.0)]

        # Run Differential Evolution with optimized parameters for speed vs quality
        ta_de_result = my_global_calibrator.calibrate(
            initial_params=my_initial_wing_model,  # this is useless for DE but kept for interface consistency
            strikes=strikes_list,
            market_volatilities=market_vols,
            market_vegas=market_vegas,
            parameter_bounds=param_bound,#ta_initial_params.get_parameter_bounds(),
            enforce_arbitrage_free=True,
            popsize=20,  # Reduced population size for faster execution
            maxiter=500,  # Reduced iterations for faster execution
            seed=42,      # For reproducible results
            weights=weight_arr
        )
        calibration_results.append(ta_de_result)
    except Exception as e:
        print(f"⚠️ DE error: {e}")
    print(f"Loss: {ta_de_result.error:.8f}, Success: {ta_de_result.success}, Message: {ta_de_result.message}, Parameters: {ta_de_result.parameters}")

🧬 Running Differential Evolution (Global Optimization)...
Loss: 0.04653915, Success: True, Message: Differential Evolution successful (nfev: 5095), Parameters: WingModelParameters(vr=0.6993, sr=0.0541, pc=0.2316, cc=0.1380, dc=-1.8114, uc=4.2853, dsm=4.6498, usm=1.6849)


In [12]:
type(calibration_results[1].parameters)

utils.volatility_fitter.wing_model.wing_model_parameters.WingModelParameters

In [15]:
pl.DataFrame([result.to_dict() for result in calibration_results])

method,success,error,message,time_elapsed,num_evaluations,vr,sr,pc,cc,dc,uc,dsm,usm
str,bool,f64,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""Initial Setup""",false,7.279213,"""Initial parameters before cali…",0.0,0,0.4991,-1.754,1.29318,1.86,-0.093005,1.4,3.0,4.87
"""SLSQP""",true,0.133905,"""Optimization successful""",0.277289,127,0.700404,0.04038,1.102421,0.164879,-0.01023,1.4,2.786554,4.87
"""L-BFGS-B""",true,0.13389,"""Optimization successful""",0.590384,270,0.700375,0.04089,1.255795,0.163846,-0.01,1.4,2.830292,4.87
"""Differential Evolution""",true,0.046539,"""Differential Evolution success…",3.097949,5095,0.699265,0.054105,0.231603,0.138013,-1.811406,4.28526,4.64979,1.684942


In [ ]:
# Visualize results - Compare all calibration methods
import numpy as np
import plotly.graph_objects as go

# Create plot
fig = go.Figure()

# Plot market data with error bars
fig.add_trace(go.Scatter(
    x=strikes_list, y=market_vols, mode='markers', name='Market IV',
    error_y=dict(type='data', symmetric=False,
        array=(df_option_with_vola_and_greeks['askVola']-df_option_with_vola_and_greeks['midVola']).to_numpy()/100,
        arrayminus=(df_option_with_vola_and_greeks['midVola']-df_option_with_vola_and_greeks['bidVola']).to_numpy()/100,
        visible=True, color='rgba(0,0,0,0.3)'
    ),
    marker=dict(size=8, color='black', symbol='circle'),
    hovertemplate='Strike: %{x}<br>Market Vol: %{y:.4f}<extra></extra>'
))

# Generate extended strike range for smooth curves
extended_strikes = np.linspace(min(strikes_list) * 0.9, max(strikes_list) * 1.1, 100)

# Plot each calibration result
subtitle_text = ""
for calibration_result in calibration_results:
    model = TimeAdjustedWingModel(calibration_result.parameters)
    model_vols = [model.calculate_volatility_from_strike(strike) for strike in extended_strikes]
    fig.add_trace(go.Scatter(
        x=extended_strikes, y=model_vols, mode='lines', name=(name:=calibration_result.optimization_method),
        line=dict(width=2, dash="dash" if name.startswith("Initial") else "solid"), opacity=0.8,
        hovertemplate=f'{name}<br>Strike: %{{x}}<br>Vol: %{{y:.4f}}<extra></extra>'))
    subtitle_text += f"{name}: Loss={calibration_result.error:.5f} &nbsp;&nbsp; "

# Forward price line
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Layout
fig.update_layout(
    title=f"Optimisation Comparison @ {snapshot_time} - {my_expiry} Expiry" +
            f"<span style='font-size:12px'>  (Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f})<br>" +
            f"<span style='font-size:10px'>{subtitle_text}</span>",
    xaxis_title='Strike Price', yaxis_title='Implied Volatility', width=1000, height=650, template='plotly_white', hovermode='closest',
    legend=dict(x=0.80, y=0.75, bgcolor='rgba(255,255,255,0.5)'),
    yaxis_range=(df_option_with_vola_and_greeks['bidVola'].min()/100*0.95, df_option_with_vola_and_greeks['askVola'].max()/100*1.15),
    xaxis=dict(range=[min(extended_strikes), max(extended_strikes)]),
    margin=dict(t=100)
)

# Add strike range lines if my_result is available
# if 'my_result' in locals() and my_result is not None and my_result.success:
#     # Extract the parameters from the best result (last one in the list before 'Initial Guess')
#     best_result_params = None
#     for name, params, _, _ in calibration_results:
#         if name != "Initial Guess":
#             best_result_params = params

#     if best_result_params:
#         model_for_ranges = TimeAdjustedWingModel(best_result_params)
#         for range_name, range_strike in model_for_ranges.get_strike_ranges().items():
#             fig.add_vline(x=range_strike, line=dict(color='gray', dash='dot', width=1))
    
fig.show()

## 5. Option Chain with Fitted Vol and Greeks

In [ ]:
# Add Fitted Volatilities and Greeks to existing DataFrame
import polars as pl
from utils.pricer.black76_option_pricer import Black76OptionPricer

# Get best model
best_result = min([r for r in calibration_results if r.optimization_method != "Initial Setup"], key=lambda x: x.error)
best_model = TimeAdjustedWingModel(best_result.parameters)
print(f"🎯 Using: {best_result.optimization_method} (Error: {best_result.error:.6f})")

# Calculate fitted data for all strikes
fitted_data = []
for i, strike in enumerate(strikes_list):
    fitted_vol = best_model.calculate_volatility_from_strike(strike)
    
    if time_to_expiry > 0 and fitted_vol > 0:
        call_greeks = Black76OptionPricer(forward_price, strike, time_to_expiry, rate, fitted_vol).get_all_greeks('call')
        put_greeks = Black76OptionPricer(forward_price, strike, time_to_expiry, rate, fitted_vol).get_all_greeks('put')
        fitted_data.append([fitted_vol*100, call_greeks['delta'], call_greeks['gamma'], call_greeks['theta'], 
                           call_greeks['vega']/100, call_greeks['price'], put_greeks['price'], (fitted_vol-market_vols[i])*100] if market_vols[i] > 0 else 0)
    else:
        fitted_data.append([fitted_vol*100, 0, 0, 0, 0, 0, 0, (fitted_vol-market_vols[i])*100, 0])

# Add to DataFrame
fitted_arrays = list(zip(*fitted_data))
df_option_with_vola_and_greeks = df_option_with_vola_and_greeks.with_columns([
    pl.Series(name, data) for name, data in zip(
        ["fitVola", "delta", "gamma", "theta", "vega", "tv_C", "tv_P", "vol_diff"], fitted_arrays)])

display(df_option_with_vola_and_greeks.select(["strike","midVola","bp0_C_usd","tv_C","ap0_C_usd","bp0_P_usd","tv_P","ap0_P_usd","fitVola","vol_diff",
                                               "delta","gamma","vega"]))

## 6. Arbitrage Detection & Visualization

In [ ]:
df_option_with_vola_and_greeks

In [ ]:
# Simplified Arbitrage Detection using pre-calculated call prices
def analyze_arbitrage_conditions(df_with_fitted_prices):
    """Quick arbitrage check using butterfly spreads from fitted call prices"""
    try:
        strikes = df_with_fitted_prices['strike'].to_list()
        call_prices = df_with_fitted_prices['fitted_call_price'].to_list()
        violations = 0
        
        # Check butterfly spreads using adjacent strikes (every 3rd for speed)
        for i in range(3, len(strikes)-3, 3):
            # Get butterfly spread: C(K1) - 2*C(K2) + C(K3)
            butterfly = call_prices[i-3] - 2*call_prices[i] + call_prices[i+3]
            if butterfly < -1e-4:  # Should be >= 0 for no arbitrage
                violations += 1
                
    except Exception:
        violations = 0
    
    # Results
    is_clean = violations == 0
    status = '✅ CLEAN' if is_clean else f'⚠️ {violations} violations'
    return {'violations': violations, 'is_arbitrage_free': is_clean, 'status': status}

print("✅ Arbitrage detection loaded (using pre-calculated prices)")

In [ ]:
# Analyze arbitrage using fitted call prices from DataFrame
print("🔍 ARBITRAGE ANALYSIS USING FITTED PRICES")
print("="*50)

# Use the enhanced DataFrame with fitted call prices
analysis = analyze_arbitrage_conditions(df_option_with_vola_and_greeks)
print(f"📈 FITTED MODEL ARBITRAGE CHECK: {analysis['status']}")
print(f"   - Violations: {analysis['violations']}")
print(f"   - Arbitrage-free: {analysis['is_arbitrage_free']}")

print(f"\n✅ Arbitrage analysis complete using {len(df_option_with_vola_and_greeks)} fitted prices")

In [ ]:
df_option_with_vola_and_greeks.write_csv("option_data.csv")

In [ ]:
df_option_with_vola_and_greeks


In [ ]:
calibration_results.to_pickle("calibration_results.pkl")

In [ ]:
import pickle

with open("calibration_results.pkl", "wb") as f:
    pickle.dump(calibration_results, f)

In [ ]:
calibration_results

In [ ]:
import pickle
with open("calibration_results.pkl", "rb") as f:
        temp = pickle.load(f)

In [ ]:
temp[0]
